In [11]:
import robin_stocks.robinhood as r
import pandas as pd
import seaborn as sns
import plotly.express as px

def authenticate_robinhood(username, password):
    """
    Logs into Robinhood account using provided username and password.
    
    Parameters:
    username (str): Robinhood account username
    password (str): Robinhood account password
    
    Returns:
    dict: Login result containing access token and other details
    """
    login_result = r.login(username, password)
    return login_result

def find_optimal_option(r, ticker):
    # Fetch current price
    quote_data = r.stocks.get_quotes(ticker)[0]
    current_price = float(quote_data['last_trade_price'])
    
    # Retrieve option chains
    chain_id = r.options.get_chains(ticker)['id']
    options = r.options.get_option_market_data_by_id(chain_id)
    
    # Build a DataFrame
    df = pd.DataFrame(options)
    print(df)
    df['strike_price'] = df['strike_price'].astype(float)
    
    # Calculate distance from current price
    df['distance'] = abs(df['strike_price'] - current_price)
    
    # Get Greek data
    market_data = [r.options.get_option_market_data_by_id(opt['id'])[0] for opt in options]
    market_df = pd.DataFrame(market_data)
    df['gamma'] = market_df['gamma'].astype(float)
    df['theta'] = market_df['theta'].astype(float)
    df['iv'] = market_df['implied_volatility'].astype(float)
    
    # Define a simple metric to find the "optimal" option
    df['score'] = (
        (1 / (1 + df['distance'])) 
        + (df['gamma'] / (1 + abs(df['theta']))) 
        - (df['iv'] / 100)
    )
    
    # Pick option with highest score
    optimal = df.loc[df['score'].idxmax()]
    
    # Create interactive scatter plot
    fig_scatter = px.scatter(df, x="distance", y="score", title="Score vs Distance")
    fig_scatter.show()

    # Create interactive correlation heatmap
    corr = df[["distance", "gamma", "theta", "iv", "score"]].corr()
    fig_heatmap = px.imshow(corr, text_auto=True, color_continuous_scale='RdBu_r', title="Correlation Heatmap")
    fig_heatmap.show()
    
    return optimal


In [ ]:

# Example usage
username = input("Enter your Robinhood username: ")
password = input("Enter your Robinhood password: ")
login_result = authenticate_robinhood(username, password)

ticker = "AAPL"


In [18]:
import datetime

def get_closest_dates(ticker):
    # Get the current date
    current_date = datetime.date.today()
    
    # Get the expiration dates from Robinhood
    options_chain = r.options.get_chains(ticker)
    expiration_dates = options_chain['expiration_dates']
    
    # Convert expiration dates to datetime.date objects
    expiration_dates = [datetime.datetime.strptime(date, '%Y-%m-%d').date() for date in expiration_dates]
    
    # Define target dates
    target_dates = {
        '1_week': current_date + datetime.timedelta(weeks=1),
        '1_month': current_date + datetime.timedelta(days=30),
        '1_year': current_date + datetime.timedelta(days=365)
    }
    
    # Function to find the closest date
    def find_closest_date(target_date):
        closest_date = min(expiration_dates, key=lambda x: abs(x - target_date), default=None)
        return closest_date if closest_date else None
    
    # Find the closest dates
    closest_dates = {key: find_closest_date(target_date) for key, target_date in target_dates.items()}
    
    return closest_dates

# Example usage
ticker = 'AAPL'
closest_dates = get_closest_dates(ticker)
print(closest_dates)

{'1_week': datetime.date(2025, 2, 7), '1_month': datetime.date(2025, 2, 28), '1_year': datetime.date(2026, 1, 16)}


In [19]:
import datetime
import robin_stocks.robinhood as r

def get_closest_dates(ticker):
    # Get the current date
    current_date = datetime.date.today()
    
    # Get the expiration dates from Robinhood
    options_chain = r.options.get_chains(ticker)
    expiration_dates = options_chain['expiration_dates']
    
    # Convert expiration dates to datetime.date objects
    expiration_dates = [datetime.datetime.strptime(date, '%Y-%m-%d').date() for date in expiration_dates]
    
    # Define target dates
    target_dates = {
        '1_week': current_date + datetime.timedelta(weeks=1),
        '1_month': current_date + datetime.timedelta(days=30),
        '1_year': current_date + datetime.timedelta(days=365)
    }
    
    # Function to find the closest date
    def find_closest_date(target_date):
        closest_date = min(expiration_dates, key=lambda x: abs(x - target_date), default=None)
        return closest_date if closest_date else None
    
    # Find the closest dates
    closest_dates = {key: find_closest_date(target_date) for key, target_date in target_dates.items()}
    
    return closest_dates

def get_option_data(ticker, expiration_date, strike_price, option_type):
    """
    Pull data for options at 5%, 10%, and 20% over/under the strike price.
    
    Parameters:
    ticker (str): The ticker symbol.
    expiration_date (str): The expiration date in 'YYYY-MM-DD' format.
    strike_price (float): The current strike price.
    option_type (str): 'call' or 'put'.
    
    Returns:
    dict: A dictionary with option data.
    """
    percentages = [0.05, 0.10, 0.20]
    option_data = {}
    
    for pct in percentages:
        if option_type == 'call':
            adjusted_strike = strike_price * (1 + pct)
        elif option_type == 'put':
            adjusted_strike = strike_price * (1 - pct)
        else:
            raise ValueError("option_type must be 'call' or 'put'")
        
        # Round the adjusted strike price to the nearest valid strike price
        adjusted_strike = round(adjusted_strike, 2)
        
        # Get option data
        options = r.options.find_options_by_expiration_and_strike(ticker, expiration_date, adjusted_strike, option_type)
        option_data[f'{int(pct*100)}%_{option_type}'] = options
    
    return option_data

# Example usage
ticker = 'AAPL'
closest_dates = get_closest_dates(ticker)

# Assuming you have a way to get the current strike price, e.g., from the stock's current price
current_strike_price = 150.00  # Example strike price

all_option_data = {}
for period, date in closest_dates.items():
    if date:
        expiration_date = date.strftime('%Y-%m-%d')
        call_data = get_option_data(ticker, expiration_date, current_strike_price, 'call')
        put_data = get_option_data(ticker, expiration_date, current_strike_price, 'put')
        all_option_data[period] = {'call': call_data, 'put': put_data}

print(all_option_data)

Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data {'1_week': {'call': {'5%_call': [], '10%_call': [{'chain_id': '7dd906e5-7d4b-4161-a3fe-2c3b62038482', 'chain_symbol': 'AAPL', 'created_at': '2025-01-03T02:04:18.778282Z', 'expiration_date': '2025-02-07', 'id': '8692cc9b-13ff-4fa1-8abd-65cf1de30a06', 'issue_date': '2025-01-03', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'tradable', 'state': 'active', 'strike_price': '165.0000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2025-01-31T20:59:48.0714304Z', 'url': 'https://api.robinhood.com/options/instruments/8692cc9b-13ff-4fa1-8abd-65cf1de30a06/', 'sellout_datetime': '2025-02-07T20:30:00+00:00', 'long_strategy_code': '8692cc9b-13ff-4fa1-8abd-65cf1de30a06_L1', 'short_strategy_code': '8692cc9b-13ff-4fa1-8abd-65cf1de30a06_S1'

In [39]:
import time
import datetime

def get_option_data(ticker, expiration_date, strike_price, option_type):
    """
    Pull data for options at 5%, 10%, and 20% over/under the strike price.
    If volume of the given option is 0, find another option by increasing the strike price by 5%,
    or increasing the price (for call) / decreasing the price (for put).
    
    Parameters:
    ticker (str): The ticker symbol.
    expiration_date (str): The expiration date in 'YYYY-MM-DD' format.
    strike_price (float): The current strike price.
    option_type (str): 'call' or 'put'.
    
    Returns:
    dict: A dictionary with option data.
    """
    percentages = [0.05, 0.10, 0.20]
    option_data = {}
    
    for pct in percentages:
        if option_type == 'call':
            adjusted_strike = strike_price * (1 + pct)
        elif option_type == 'put':
            adjusted_strike = strike_price * (1 - pct)
        else:
            raise ValueError("option_type must be 'call' or 'put'")
        
        # Round the adjusted strike price to the nearest multiple of 5
        adjusted_strike = round(adjusted_strike / 5) * 5
        
        while True:
            # Get option data
            print(f"Fetching options for {ticker}, expiration: {expiration_date}, strike: {adjusted_strike}, type: {option_type}")
            options = r.options.find_options_by_expiration_and_strike(ticker, expiration_date, adjusted_strike, option_type)
            time.sleep(2)  # Sleep for 2 seconds before each call
            
            if options:
                sample_response = options[0] if isinstance(options, list) and options else options
                print(f"Sample response: {sample_response}")
                
                # Check if volume is 0
                if all(opt['volume'] == 0 for opt in options if isinstance(opt, dict)):
                    # Adjust strike price by $5
                    if option_type == 'call':
                        adjusted_strike += 5  # Increase the price for call
                    elif option_type == 'put':
                        adjusted_strike -= 5  # Decrease the price for put
                    print(f"Volume is 0, adjusting strike price to {adjusted_strike}")
                else:
                    option_data[f'{int(pct*100)}%_{option_type}'] = options
                    break
            else:
                print("No options found, adjusting expiration date")
                expiration_date = (datetime.datetime.strptime(expiration_date, '%Y-%m-%d') + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        
    return option_data

# Example usage
for period, date in closest_dates.items():
    if date:
        expiration_date = date.strftime('%Y-%m-%d')
        call_data = get_option_data(ticker, expiration_date, current_strike_price, 'call')
        put_data = get_option_data(ticker, expiration_date, current_strike_price, 'put')
        all_option_data[period] = {'call': call_data, 'put': put_data}

print(all_option_data)


Fetching options for AAPL, expiration: 2025-02-07, strike: 240, type: call
Loading Market Data Sample response: {'chain_id': '7dd906e5-7d4b-4161-a3fe-2c3b62038482', 'chain_symbol': 'AAPL', 'created_at': '2024-12-25T02:07:16.718755Z', 'expiration_date': '2025-02-07', 'id': '567edcc2-1a7d-46e2-977e-4c2581ea7f5c', 'issue_date': '2024-12-26', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'tradable', 'state': 'active', 'strike_price': '240.0000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2025-01-31T20:59:59.933100032Z', 'url': 'https://api.robinhood.com/options/instruments/567edcc2-1a7d-46e2-977e-4c2581ea7f5c/', 'sellout_datetime': '2025-02-07T20:30:00+00:00', 'long_strategy_code': '567edcc2-1a7d-46e2-977e-4c2581ea7f5c_L1', 'short_strategy_code': '567edcc2-1a7d-46e2-977e-4c2581ea7f5c_S1', 'underlying_type': 'equity', 'adjusted_mark_price': '1.770000', 'adjusted_mark_price_round_down': '1.770000', 'ask_price': '1.790000'

In [40]:
all_option_data

{'1_week': {'call': {'5%_call': [{'chain_id': '7dd906e5-7d4b-4161-a3fe-2c3b62038482',
     'chain_symbol': 'AAPL',
     'created_at': '2024-12-25T02:07:16.718755Z',
     'expiration_date': '2025-02-07',
     'id': '567edcc2-1a7d-46e2-977e-4c2581ea7f5c',
     'issue_date': '2024-12-26',
     'min_ticks': {'above_tick': '0.05',
      'below_tick': '0.01',
      'cutoff_price': '3.00'},
     'rhs_tradability': 'tradable',
     'state': 'active',
     'strike_price': '240.0000',
     'tradability': 'tradable',
     'type': 'call',
     'updated_at': '2025-01-31T20:59:59.933100032Z',
     'url': 'https://api.robinhood.com/options/instruments/567edcc2-1a7d-46e2-977e-4c2581ea7f5c/',
     'sellout_datetime': '2025-02-07T20:30:00+00:00',
     'long_strategy_code': '567edcc2-1a7d-46e2-977e-4c2581ea7f5c_L1',
     'short_strategy_code': '567edcc2-1a7d-46e2-977e-4c2581ea7f5c_S1',
     'underlying_type': 'equity',
     'adjusted_mark_price': '1.770000',
     'adjusted_mark_price_round_down': '1.7700

In [30]:
def create_option_plotly(all_option_data):
    """
    Convert the nested Robinhood option data into a DataFrame, rank it,
    and return a Plotly figure along with the ranked DataFrame.
    Focus on 'mark_price', 'chance_of_profit_long', 'gamma', etc.
    """
    records = []
    for period, cp_data in all_option_data.items():
        for call_put, pct_data in cp_data.items():
            for pct_key, options_list in pct_data.items():
                for opt in options_list:
                    if not isinstance(opt, dict):
                        continue
                    record = {
                        'period': period,
                        'pct_key': pct_key,
                        'type': call_put,  # 'call' or 'put'
                        'mark_price': opt.get('mark_price'),
                        'chance_of_profit_long': opt.get('chance_of_profit_long'),
                        'chance_of_profit_short': opt.get('chance_of_profit_short'),
                        'delta': opt.get('delta'),
                        'gamma': opt.get('gamma'),
                        'implied_volatility': opt.get('implied_volatility'),
                        'rho': opt.get('rho'),
                        'theta': opt.get('theta'),
                        'vega': opt.get('vega'),
                        'symbol': opt.get('chain_symbol'),
                        'strike_price': opt.get('strike_price'),
                        'expiration_date': opt.get('expiration_date')
                    }
                    records.append(record)

    df = pd.DataFrame(records)

    # Drop rows with missing data you care about (optional)
    df.dropna(subset=['mark_price', 'chance_of_profit_long', 'gamma'], inplace=True)

    # Convert mark_price and other metrics to numeric
    df['mark_price'] = pd.to_numeric(df['mark_price'], errors='coerce')
    df['chance_of_profit_long'] = pd.to_numeric(df['chance_of_profit_long'], errors='coerce')
    df['chance_of_profit_short'] = pd.to_numeric(df['chance_of_profit_short'], errors='coerce')
    df['delta'] = pd.to_numeric(df['delta'], errors='coerce')
    df['gamma'] = pd.to_numeric(df['gamma'], errors='coerce')
    df['implied_volatility'] = pd.to_numeric(df['implied_volatility'], errors='coerce')
    df['rho'] = pd.to_numeric(df['rho'], errors='coerce')
    df['theta'] = pd.to_numeric(df['theta'], errors='coerce')
    df['vega'] = pd.to_numeric(df['vega'], errors='coerce')

    # Rank by chance_of_profit_long (descending)
    df['rank'] = df['chance_of_profit_long'].rank(ascending=False)
    df.sort_values('rank', inplace=True)

    # Create an interactive Plotly scatter plot
    fig = px.scatter(
        df,
        x='gamma',
        y='chance_of_profit_long',
        size='mark_price',
        color='type',
        hover_data=['period', 'pct_key', 'strike_price', 'expiration_date', 'chance_of_profit_short', 'delta', 'implied_volatility', 'rho', 'theta', 'vega'],
        title='Option Comparison'
    )

    return fig, df

fig, df = create_option_plotly(all_option_data)

In [28]:
df

,period,pct_key,type,mark_price,chance_of_profit_long,chance_of_profit_short,delta,gamma,implied_volatility,rho,theta,vega,symbol,strike_price,rank
3,1_month,10%_call,call,71.025,1.000000,0.000000,1.000143,-0.000009,0.000000,0.132612,-0.021212,-8222.318818,AAPL,165.0000,1.0
1,1_week,20%_call,call,56.100,0.475599,0.524401,0.990356,0.000919,0.880733,0.032332,-0.075977,0.008220,AAPL,180.0000,2.0
4,1_month,20%_call,call,56.650,0.473766,0.526234,0.983910,0.001299,0.478849,0.132879,-0.043899,0.026110,AAPL,180.0000,3.0
0,1_week,10%_call,call,71.100,0.468626,0.531374,0.991614,0.000624,1.146421,0.029654,-0.082449,0.007273,AAPL,165.0000,4.0
6,1_year,10%_call,call,79.750,0.444675,0.555325,0.930815,0.001893,0.304505,1.339145,-0.029347,0.307172,AAPL,165.0000,5.0
7,1_year,20%_call,call,66.975,0.434644,0.565356,0.889997,0.002784,0.292584,1.369206,-0.034491,0.434084,AAPL,180.0000,6.0
8,1_year,10%_put,put,1.175,0.063151,0.936849,-0.031675,0.000853,0.363216,-0.080566,-0.007571,0.164258,AAPL,135.0000,7.0
9,1_year,20%_put,put,0.800,0.047380,0.952620,-0.020728,0.000542,0.395849,-0.053346,-0.005885,0.115202,AAPL,120.0000,8.0
2,1_week,20%_put,put,0.005,0.001419,0.998581,-0.000680,0.000042,1.620278,-0.000031,-0.009136,0.000752,AAPL,120.0000,9.5
5,1_month,20%_put,put,0.010,0.001419,0.998581,-0.000678,0.000046,0.797436,-0.000129,-0.002183,0.001528,AAPL,120.0000,9.5


In [29]:
fig.show()

In [12]:
optimal_option = find_optimal_option(r, ticker)
print(optimal_option)

404 Client Error: Not Found for url: https://api.robinhood.com/options/instruments/7dd906e5-7d4b-4161-a3fe-2c3b62038482/
Empty DataFrame
Columns: []
Index: []


KeyError: 'strike_price'